In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# https://archive.ics.uci.edu/dataset/59/letter+recognition
# first used in "Letter Recognition Using Holland-Style Adaptive Classifiers"
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
letter_recognition = fetch_ucirepo(id=59) 
  
# data (as pandas dataframes) 
X_data = letter_recognition.data.features 
y_data = letter_recognition.data.targets

In [ ]:
X = X_data.to_numpy()
y = y_data.to_numpy()

In [ ]:
def map_letter_array_to_numbers(letters):
    letter_to_number = {chr(i): i - ord('A') + 1 for i in range(ord('A'), ord('Z') + 1)}
    return np.array([letter_to_number.get(letter, 0) for letter in letters])

def map_number_array_to_letters(numbers):
    number_to_letter = {int(c): chr(c + ord('A') - 1) for c in range(26)}
    return np.array([number_to_letter.get(number, 'A') for number in numbers])    

In [ ]:
training_number = 16000
x_train = X[:training_number]
x_test = X[training_number:]
y_train_letters = y[:training_number].flatten()
y_test_letters = y[training_number:].flatten()
y_train = map_letter_array_to_numbers(y_train_letters)
y_test = map_letter_array_to_numbers(y_test_letters)

In [ ]:
print(x_train.shape)
print(x_train[0])
print(y_train_letters[0])
print(y_train[0])

In [ ]:
model = Sequential(
    [
        tf.keras.Input(shape=(16,)),
        Dense(100, activation = 'relu', name = 'L1'),
        Dense(60, activation = 'relu', name = 'L2'),
        Dense(27, activation = 'linear', name = 'L3')
    ], name = "letter_recognition_model"
)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=40,
    validation_data=(x_test, y_test),
)

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1, figsize = (4,3))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
ax.plot(history.history['loss'], label='loss')
ax.set_ylim([0, 2])
ax.set_xlabel('Epoch')
ax.set_ylabel('loss (cost)')
ax.legend()
ax.grid(True)
plt.show()

In [ ]:
loss, acc = model.evaluate(x_test, y_test)

In [ ]:
model.save('letter_recognition_model.h5')

In [ ]:
prediction = model.predict(x_test[:1])
yhat = np.argmax(prediction)
print(yhat)
print(y_test[0])